# 2003-06-24T05:47:09

# Quadrant Four
Try to reduce it using esorex. The online request ESO archive data service list the require calibrations file:

![Example complete dataset](images/quad4.png)


In [1]:
import glob
import os
import re
import itertools
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
import numpy as np

def flatten(listOfLists):
    "Flatten one level of nesting"
    return itertools.chain.from_iterable(listOfLists)

In [2]:
datadir = '/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/'



## Category

Any raw or product frame can be classified on the basis of a set of keywords read from its header. Data classification is based on FITS keyword values. In the case of raw frames, classification can be defined by looking at least at three keyword values: DPR TYPE, DPR CATG, and DPR TECH. In the case of data products, the classification of the frame is stored into the hierarchical keyword PRO CATG.


**DPR TYPE = FLAT,LAMP**

**DO categoy: IFU_SCREEN_FLAT**

Processed by: vmifucalib

**DPR TYPE = WAVE,LAMP**

**DO category: IFU_ARC_SPECTRUM**

Processed by: vmifucalib


## Science

**DO category: IFU_SCIENCE**

DPR CATG = SCIENCE

DPR TYPE = OBJECT

# Create the SOF files


First specify all the rawdata for the specific science frame

In [3]:
rawdata = {'IFU_SCIENCE':'VIMOS.2003-06-24T05:47:09.496.fits',
           'LINE_CATALOG':'M.VIMOS.2008-06-17T18:11:31.782.fits',
          'EXTINCT_TABLE':'M.VIMOS.2008-06-17T18:12:53.176.fits',
          'STD_FLUX_TABLE':'M.VIMOS.2008-06-17T18:13:13.956.fits',
           'IFU_IDENT':'M.VIMOS.2008-06-17T18:16:12.090.fits',
           'IFU_MASTER_RESPONSE':'M.VIMOS.2008-06-17T18:20:50.827.fits',
           'BIAS':['VIMOS.2003-06-23T14:48:34.460.fits','VIMOS.2003-06-23T14:49:26.873.fits','VIMOS.2003-06-23T14:50:19.473.fits','VIMOS.2003-06-23T14:51:12.022.fits','VIMOS.2003-06-23T14:52:04.694.fits',
                  'VIMOS.2003-06-24T11:27:11.125.fits','VIMOS.2003-06-24T11:28:03.628.fits','VIMOS.2003-06-24T11:28:56.487.fits','VIMOS.2003-06-24T11:29:49.974.fits','VIMOS.2003-06-24T11:30:43.187.fits'],
           'IFU_ARC_SPECTRUM':'VIMOS.2003-06-24T06:32:58.133.fits',
           'IFU_STANDARD':'VIMOS.2003-06-24T10:38:01.522.fits',
           'IFU_SCREEN_FLAT':['VIMOS.2003-06-24T10:43:31.637.fits','VIMOS.2003-06-24T10:44:42.777.fits','VIMOS.2003-06-24T10:45:54.169.fits']
          }

Then we add the directory for each of the data

In [4]:
for k,v in rawdata.items():
    try:
        rawdata[k] = datadir+v
    except:
        temp = [datadir+i for i in v]
        rawdata[k] = temp

t = fits.open(rawdata['IFU_SCIENCE'])[0]
quadrant = t.header['HIERARCH ESO OCS CON QUAD']
print(quadrant)


4


## Create bias.sof

In [5]:
#limit = 5
outdir = datadir+'quadrant'+str(quadrant)

#Create outdir if it doesnt exist
if not os.path.exists(outdir):
    os.makedirs(outdir)


filenamebias = outdir+'/bias.sof'
with open(filenamebias,'w') as file:
    for bias in rawdata['BIAS'][0:]:
        cat = 'BIAS'
        string = '{} {}\n'.format(bias,cat)
        file.write(string)

Call the vmbias routine and give the path to esorex


In [6]:
stackmethod = '--StackMethod=Average'
esorexpath = '/bin/esorex'
routine = 'vmbias'
outputdir = '--output-dir={}'.format(outdir)
logdir = '--log-dir={}'.format(outdir)



In [7]:
commandbias = '{} {} {} {}  {}'.format(esorexpath,outputdir,logdir,routine,  filenamebias)
print(commandbias)
os.system(commandbias)

/bin/esorex --output-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4 --log-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4 vmbias  /home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4/bias.sof


0

Add the master_bias to rawdata dictionary

In [8]:
rawdata['MASTER_BIAS'] = outdir+'/master_bias.fits'

## Create calib.sof

In [9]:
inputfilescalib = ['IFU_SCREEN_FLAT','IFU_ARC_SPECTRUM','LINE_CATALOG','IFU_IDENT','MASTER_BIAS']

filenamecalib = outdir+'/calib.sof'
with open(filenamecalib,'w') as file:
    for i in inputfilescalib:
        tmpfile = rawdata[i]
        if isinstance(tmpfile, list):    
            for j in tmpfile:
                string = '{} {}\n'.format(j,i)
                file.write(string)
        else:
            string = '{} {}\n'.format(tmpfile,i)
            file.write(string)

            
# Define the routine and call it
routine = 'vmifucalib'

commandcalib = '{} {} {} {}  {}'.format(esorexpath,outputdir,logdir,routine,  filenamecalib)
print(commandcalib)
os.system(commandcalib)

/bin/esorex --output-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4 --log-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4 vmifucalib  /home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4/calib.sof


0

In [10]:
rawdata['IFU_IDS'] = outdir+'/ifu_ids.fits'
rawdata['IFU_TRACE'] = outdir+'/ifu_trace.fits'
rawdata['IFU_TRANSMISSION'] = outdir+'/ifu_transmission.fits'

## Create standard.sof

In [11]:
inputfilesstandard = ['IFU_STANDARD','MASTER_BIAS','IFU_IDS','IFU_TRACE','IFU_TRANSMISSION','EXTINCT_TABLE','STD_FLUX_TABLE']

filenamestandard = outdir+'/ifustandard.sof'
with open(filenamestandard,'w') as file:
    for i in inputfilesstandard:
        tmpfile = rawdata[i]
        if isinstance(tmpfile, list):    
            for j in tmpfile:
                string = '{} {}\n'.format(j,i)
                file.write(string)
        else:
            string = '{} {}\n'.format(tmpfile,i)
            file.write(string)


In [12]:
routine = 'vmifustandard'

commandstandard = '{} {} {} {}  {}'.format(esorexpath,outputdir,logdir,routine,  filenamestandard)
print(commandstandard)
os.system(commandstandard)

/bin/esorex --output-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4 --log-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4 vmifustandard  /home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4/ifustandard.sof


0

## Create science.sof

In [13]:
rawdata['IFU_SPECPHOT_TABLE'] = outdir+'/ifu_specphot_table.fits'

inputfilesscience = ['IFU_SCIENCE','MASTER_BIAS','IFU_IDS','IFU_TRACE','IFU_TRANSMISSION','EXTINCT_TABLE','IFU_SPECPHOT_TABLE']

filenamescience = outdir+'/ifuscience.sof'
with open(filenamescience,'w') as file:
    for i in inputfilesscience:
        tmpfile = rawdata[i]
        if isinstance(tmpfile, list):    
            for j in tmpfile:
                string = '{} {}\n'.format(j,i)
                file.write(string)
        else:
            string = '{} {}\n'.format(tmpfile,i)
            file.write(string)

In [14]:
routine = 'vmifuscience'

calibrateflux = '--CalibrateFlux=true'

commandscience = '{} {} {} {} {} {}'.format(esorexpath,outputdir,logdir,routine, calibrateflux ,filenamescience)
print(commandscience)
os.system(commandscience)

/bin/esorex --output-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4 --log-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4 vmifuscience --CalibrateFlux=true /home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B5/2003-06-24T05:47:09/RawESOdata/quadrant4/ifuscience.sof


0